In [ ]:

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import progressbar
import catboost
import joblib
from catboost import CatBoostClassifier
from catboost import Pool
from catboost import MetricVisualizer
from sklearn.preprocessing import LabelEncoder
import timeit

from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, \
recall_score, f1_score
from sklearn.model_selection import ParameterGrid

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, GroupShuffleSplit
import seaborn as sns

%load_ext autoreload
%autoreload 2



In [ ]:
df = joblib.load('../data/interim/transcript.joblib')
df.head(100)

In [ ]:
df = joblib.load('../data/interim/transcript_final_optimised.joblib')
df

In [ ]:
df.gender.value_counts()

In [ ]:
separation = {'completed': 1,
              'not_completed': 2
             }
label_grid = separation

In [ ]:
df['completed'] =(df['offer completed'] == 1) * label_grid['completed']

In [ ]:
df['not_completed'] =(df['offer completed'] == 0) * label_grid['not_completed']

In [ ]:
df['label'] = df[['completed', 'not_completed']].sum(axis=1)

In [ ]:
df['label']

In [ ]:
df.drop(['offer completed', 'completed', 'not_completed'], axis=1, inplace = True)

In [ ]:
cat_features = [0,4,5,92,93,94,95,96,97]

In [ ]:
df

In [ ]:
cat_features_name = ['person', 'gender', 'difficulty', 'rewarded']

for x in cat_features_name:

    df[x] = df[x].astype(str) 

In [ ]:
X = df.drop('label', axis=1)
X= X[['person', 'age', 'income', 'became_member_on', 'gender', 'rewarded',
           'difficulty', 'duration', 'bogo',
            'informational']]


X

In [ ]:
# Assigns columns index location of categorical features
cat_features = [X.columns.get_loc(i) for i in cat_features_name if i in X.columns]

cat_features

In [ ]:
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, 
                                                        random_state=42)



In [ ]:
weights = [df.label.value_counts().sum() / df.label.value_counts()[i] for i in 
               set(separation.values())]


weights


In [ ]:
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [ ]:
 model = CatBoostClassifier(
        iterations=5000,
        loss_function='MultiClass',
        early_stopping_rounds=10,
        cat_features=cat_features,
        class_weights= weights,
        verbose=100)  

In [ ]:
 model.fit(train_pool,
          eval_set=test_pool,
          verbose=100,
          plot=True);